# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [6]:
pip install scikit-learn==0.24.2

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

In [8]:

sk.__version__

'0.24.2'

### Importar o dataset

In [9]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [11]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [15]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6, activation='selu'))

### Adding the second hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [18]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [19]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [20]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7950
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4757 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4491 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4364 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4301 - accuracy: 0.7956
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.7991
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4234 - accuracy: 0.8100
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.8167
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4180 - accuracy: 0.8198
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4159 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [21]:
y_pred = ann.predict(X_test)
y_pred

array([[0.29190135],
       [0.37630785],
       [0.12010181],
       ...,
       [0.4336266 ],
       [0.26352492],
       [0.24236357]], dtype=float32)

In [22]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1499   96]
 [ 188  217]]


---
# Avaliando o Modelo

In [24]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [50]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'elu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'elu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [53]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4977 - accuracy: 0.8047
Epoch 2/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4295 - accuracy: 0.8103
Epoch 3/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4265 - accuracy: 0.8136
Epoch 4/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4235 - accuracy: 0.8155
Epoch 5/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4186 - accuracy: 0.8208
Epoch 6/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4132 - accuracy: 0.8244
Epoch 7/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4081 - accuracy: 0.8269
Epoch 8/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4032 - accuracy: 0.8300
Epoch 9/10
640/640 [==============================] - 1s 1ms/step - loss: 0.3986 - accuracy: 0.8288
Epoch 10/10
160/160 [==============================] - 0s 1ms/step - loss: 0.3935 - accuracy: 0.8300

In [54]:
print(mean,variance)

0.8428749918937684 0.007088733706019391


# Melhorando o modelo

In [55]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [56]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/10
640/640 [==============================] - 2s 1ms/step - loss: 0.4849 - accuracy: 0.7978
Epoch 2/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4350 - accuracy: 0.7980
Epoch 3/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4314 - accuracy: 0.7980
Epoch 4/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4321 - accuracy: 0.7980
Epoch 5/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4339 - accuracy: 0.7980
Epoch 6/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4327 - accuracy: 0.7980
Epoch 7/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4323 - accuracy: 0.7980
Epoch 8/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4326 - accuracy: 0.7980
Epoch 9/10
640/640 [==============================] - 1s 1ms/step - loss: 0.4318 - accuracy: 0.7980
Epoch 10/10
160/160 [==============================] - 0s 1ms/step - loss: 0.4313 - accuracy: 0.7881

In [57]:
print(mean,variance)

0.8201249957084655 0.017595285700967446


# Tuning the ANN

In [ ]:
''' from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)
### '''

In [ ]:
'''%%time

grid_search = grid_search.fit(X_train, y_train)'''

In [ ]:
'''print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)'''

# Halving Grid Search

In [35]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32, 64],
    'epochs': [50, 100, 150],
    'optimizer': ['adam', 'rmsprop', 'nadam', 'ftrl', 'adadelta']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [36]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

Epoch 1/150
800/800 [==============================] - 2s 2ms/step - loss: 0.4879 - accuracy: 0.7956
Epoch 2/150
800/800 [==============================] - 1s 1ms/step - loss: 0.4371 - accuracy: 0.7943
Epoch 3/150
800/800 [==============================] - 1s 1ms/step - loss: 0.4367 - accuracy: 0.7930
Epoch 4/150
800/800 [==============================] - 1s 2ms/step - loss: 0.4383 - accuracy: 0.7984
Epoch 5/150
800/800 [==============================] - 1s 2ms/step - loss: 0.4340 - accuracy: 0.7962
Epoch 6/150
800/800 [==============================] - 1s 2ms/step - loss: 0.4346 - accuracy: 0.7944
Epoch 7/150
800/800 [==============================] - 1s 2ms/step - loss: 0.4333 - accuracy: 0.7962
Epoch 8/150
800/800 [==============================] - 1s 2ms/step - loss: 0.4357 - accuracy: 0.7966
Epoch 9/150
800/800 [==============================] - 1s 2ms/step - loss: 0.4359 - accuracy: 0.7979
Epoch 10/150
800/800 [==============================] - 1s 1ms/step - loss: 0.4344 - accura

In [37]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 150, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.84918648310388


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)